In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [14]:
metadata = pd.read_csv('Data/metadata.csv')
ratings_train = pd.read_csv('Data/ratings-train.csv')
ratings_valid = pd.read_csv('Data/ratings-valid.csv').sample(frac=0.2, random_state=17)

In [9]:
metadata.sample()

,itemid,title,genres,country,running_min
2598,75497,또 하나의 약속 (2013),드라마,한국,120


In [15]:
ratings_train.sample()

,userid,itemid,rating
18743,MkM2Mg==,44451,6


In [16]:
ratings_valid.sample()

,userid,itemid,rating
3166,NmpTcg==,13247,10


# RMSE 함수

In [32]:
def rmse(expected, answer):
    merged = pd.merge(answer, expected, how='left', on=['userid', 'itemid'])
    merged['rating_y'] = merged['rating_y'].fillna(0)
    merged['square_error'] = (merged['rating_x'] - merged['rating_y']) ** 2
    return merged['square_error'].mean() ** 0.5

# Jaccard Similarity
아래 코드가 계산하는데 오래걸려서 수식을 수정

$u, v -> len(i_dict[u] & i_dict[v]) / len(i_dict[u] | i_dict[v])$

In [23]:
all_users = ratings_train['userid'].unique()

i_dict = {u : set(ratings_train[ratings_train['userid'] == u]['itemid'])
         for u in all_users}

def sim(u, v):
    i_u = i_dict[u]
    i_v = i_dict[v]
    
    cup = i_dict[u] | i_dict[v]
    cap = i_dict[u] & i_dict[v]
    
    if len(cup) == 0:
        return 0.0
    return len(cap) / len(cup)

In [25]:
sim('TERhUA==', 'Q1ladXM=')

0.08333333333333333

# Similar User 찾기
k 값을 5로 놓는건 이유없음. 바꿔봐도 된다. k 가 성능 결정할 수 있다

In [49]:
def similar_users(u, k):
    sims = [(sim(u, v), v) for v in all_users if u != v]
    sorted_sims = sorted(sims, reverse=True)
    topk_sims = sorted_sims[:k]
    
    topk_users = [v for s, v in topk_sims]
    
    return pd.DataFrame(topk_users, columns=['userid'])

In [50]:
similar_users('TERhUA==', 5)

,userid
0,YzkyQQ==
1,NGdmcVQ=
2,M2hETGQ=
3,V0NyaQ==
4,QTB5d0E=


In [55]:
# sim 도 함께 반환하도록 함수 수정

def similar_users(u, k):
    sims = [(sim(u, v), v) for v in all_users if u != v]
    sorted_sims = sorted(sims, reverse=True)
    topk_sims = sorted_sims[:k]
    
    return pd.DataFrame(topk_sims, columns=['sim', 'userid'])

In [56]:
similar_users('TERhUA==', 5)

,sim,userid
0,0.142857,YzkyQQ==
1,0.125000,NGdmcVQ=
2,0.125000,M2hETGQ=
3,0.111111,V0NyaQ==
4,0.111111,QTB5d0E=


# Predict

In [33]:
def predict(u, i):
    return 8

expected = ratings_valid.copy()
expected['rating'] = expected.apply(
    lambda x: predict(x['userid'], x['itemid']), axis=1)

rmse(expected, ratings_valid)

2.7423239390279464

# Predict 함수 수정
- 취향이 비슷한 사람들 중 아무도 예측에 필요한 영화를 보지 않았을 때 평점이 없기 때문에 0이 나온다
- 0이 나오지 않게 하기 위해
- 유저 평균과 유저 평균과의 차이의 차이를 따로 구해서 더해준다

In [38]:
# rating_mean = pd.DataFrame(ratings_train.groupby('userid')['rating'].mean())
rating_mean = ratings_train.groupby('userid')['rating'].mean()
rating_mean.sample()

userid
OHlIaDU=    6.5
Name: rating, dtype: float64

In [39]:
rating_mean['MzVDMXM=']

3.0

In [60]:
similar_users('TERhUA==', 5)['sim']

0    0.142857
1    0.125000
2    0.125000
3    0.111111
4    0.111111
Name: sim, dtype: float64

In [71]:
# 직접 해보기
def predict(u, i):
    
    # 해당 유저의 평점 평균 
    u_mean = rating_mean[u]
    
    # 비슷한 유저와 sim 값
    sim = similar_users(u, 5)
    
    # 해당 영화 i 의 비슷한 유저sim_users 의 평점
        # 해당영화의 평점만 불러오기
    i_rating = rating_train[rating_train['itemid'] == i][['userid', 'rating']]
        # 비슷한 유저의 평점, 해당 영화를 안 봤다면 0으로 바꿈
    sim_rating = [i_rating[i_rating['userid'] == x]['rating'] if x in i_rating['userid'] else 0 for x in sim['userid']]
    ##### 비슷한 유저가 해당 영화를 안 봤다면 다음으로 유사한 유저를 찾기###
    ###
    ###
    
    
    # 해당 영화 i 의 비슷한 유저sim_users 의 sim
    
#     -> zip
    return u_mean + np.mean([s * (r - u_mean) for s, r in zip(sim['sim'], sim_rating)])
    
expected = ratings_valid.copy()
expected['rating'] = expected.apply(
    lambda x: predict(x['userid'], x['itemid']), axis=1)

rmse(expected, ratings_valid)

2.4540348034876933

In [ ]:
# 선생님 코드

r_mean = ratings_train.groupby('userid')['rating'].mean().reset_index()
def predict(u, i):
    topk = similar_users(u, 5)
    topk['sim'] = topk.apply(lambda x: sim(u, x['userid']), axis=1)
    joined = pd.merge(topk, ratings_train[ratings_train['itemid'] == i], on='userid')
    joined = pd.merge(joined, r_mean, on='userid')
    joined['score'] = joined['sim'] * (joined['rating_x'] - joined['rating_y'])
    
    mean_u = r_mean[r_mean['userid'] == u]['rating'].iloc[0]
    
    sim_sum = joined['sim'].sum()
    
    r_ui = mean_u
    
    if len(joined) > 0:
        r_ui += joined['score'].sum() / sim_sum
    
    return r_ui
    
expected = ratings_valid.copy()
expected['rating'] = expected.apply(
    lambda x: predict(x['userid'], x['itemid']), axis=1)

rmse(expected, ratings_valid)

In [104]:
# 선생님 코드 따라 만드는 중

u = 'TERhUA=='
i = 96644

r_mean = ratings_train.groupby('userid')['rating'].mean().reset_index()

def predict(u, i):
    topk = similar_users(u, 5)

    topk['sim'] = topk.apply(lambda x: sim(u, x['userid']), axis=1)
    joined = pd.merge(topk, ratings_train[ratings_train['itemid'] == i], on='userid')
    joined = pd.merge(joined, r_mean, on='userid')

    joined['score'] = joined['sim'] * (joined['rating_x'] - joined['rating_y'])

    mean_u = r_mean[r_mean['userid'] == u]['rating'].iloc[0]

    sim_sum = joined['sim'].sum()

    r_ui = mean_u

    if len(joined) > 0:
        r_ui += joined['score'].sum() / sim_sum

    return r_ui

expected = ratings_valid.copy()
expected['rating'] = expected.apply(
    lambda x: predict(x['userid'], x['itemid']), axis=1)

rmse(expected, ratings_valid)

2.2450461039661285

# Surprise 라이브러리의 User-User CF 실습

In [85]:
from surprise import Reader, Dataset

In [89]:
# Reader 데이터 정의
# rating_scale 우리가 가지고 있는 rating 의 범위
reader = Reader(rating_scale=(0, 10))

# load_from_df(DataFrame, 기준)
train_ds = Dataset.load_from_df(ratings_train, reader).build_full_trainset()

train_ds

In [105]:
from surprise import KNNBasic, KNNWithMeans, SVD

### KNNBasic

In [108]:
model = KNNBasic(k=5)
model.fit(train_ds)

Computing the msd similarity matrix...
Done computing similarity matrix.


### KNNWithMeans 
평균에서 얼마나 차이나는지 Std (오늘 함수 구현한 5번)

In [101]:
model = KNNWithMeans(k=5)
model.fit(train_ds)

Computing the msd similarity matrix...
Done computing similarity matrix.


### model.predict(u, i).est 확인

In [92]:
u, i

('TERhUA==', 96644)

In [98]:
model.predict(u, i)

Prediction(uid='TERhUA==', iid=96644, r_ui=None, est=9.027589545014523, details={'actual_k': 5, 'was_impossible': False})

In [97]:
# est: estimation

model.predict(u, i).est

9.027589545014523

### SVD

In [110]:
model = SVD(n_epochs=200, random_state=17)
model.fit(train_ds)

In [111]:
def predict(u, i):
    return model.predict(u, i).est

expected = ratings_valid.copy()
expected['rating'] = expected.apply(
lambda x: predict(x['userid'], x['itemid']), axis=1)

rmse(expected, ratings_valid)

1.966236860523171

### SVD parameter 찾기
#### n_factors
- 전체 학습 횟수 (default 20 적다)

#### n_epochs
- 복잡한 모델일수록 요구되는 epochs 이 높다
- factor 를 높일수록 똑똑한 모델

#### random_state

#### reg_all 
- Regularization $λ$ 값이 커질수록 설명력이 줄어든다
- Objective function 에서 $λ$

> #### reg_@@@@ parameter
> - Surprise 는 $λ$ 를 각각 따로 설정해줄 수 있다 
> - 들은 $λ$ 를 설정하는 방식을 정해주는 것

#### biased
유저나 아이템에 대한 선입견이 있다면 True (Default: True)

#### lr_all
- 모든 parameter 에 대해 똑같은 Learning Rate 를 쓰는 것
> Learning Rate : 얼마나 큰 보폭으로 움직일지(최저점 찾기)
>
> 숫자가 클수록 처리속도 빠르다

- Gradiant Distinct model

In [113]:
# verbose 얼마나 반복하고 있는지 출력해준다, 거의 모든 함수에 내장

model = SVD(random_state=17, n_factors=200, n_epochs=200, lr_all=0.0052, reg_all=0.04, verbose=True)
model.fit(train_ds)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

In [114]:
def predict(u, i):
    return model.predict(u, i).est

expected = ratings_valid.copy()
expected['rating'] = expected.apply(
lambda x: predict(x['userid'], x['itemid']), axis=1)

rmse(expected, ratings_valid)

1.9359638225728153

# 유사 엔티티 분석

In [116]:
# (영화갯수, n_factors)

model.qi.shape

(5834, 200)

In [118]:
df_qi = pd.DataFrame(model.qi)

df_qi.sample()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1510,-0.338714,-0.186605,0.110693,-0.22318,0.365912,-0.004611,0.027702,-0.215159,0.303408,-0.523358,...,-0.356361,0.134025,-0.173365,0.179754,0.178434,0.100316,-0.274203,-0.081716,-0.186888,-0.000701


#### tsv 파일로 저장

In [136]:
df_qi.to_csv('Data/qi.tsv', sep='\t', index=False, header=False)

저장한 파일을 Embedding Projector website 에서 불러와서 3차원으로 변환

In [125]:
# itemid 값 확인
metadata['itemid']

0        55396
1        42858
2       121058
3         4285
4        36948
5          150
6         2886
7        66092
8         2328
9         2034
10        2361
11       11722
12       50245
13       41156
14       40535
15       44083
16       46266
17       79167
18       56972
19       51404
20       79172
21       42388
22       44949
23       96071
24       67335
25        3096
26       95915
27         136
28       42907
29       51337
         ...  
5804     69030
5805    112322
5806    100250
5807    129334
5808     19340
5809     43545
5810     45023
5811     42900
5812     94930
5813     52922
5814     93626
5815    104179
5816     84102
5817     79223
5818    126030
5819     12546
5820    119915
5821    103356
5822     90368
5823    111316
5824     68491
5825    103725
5826     83317
5827     96914
5828    129134
5829    130993
5830    134894
5831     87253
5832    127153
5833     41708
Name: itemid, Length: 5834, dtype: int64

- itemid 를 찍어보면 연속된 정수가 아니다
- 아래의 작업을 하려면 itemid 가 연속적인 정수여야 한다
- 그것이 Matrix 의 정의

In [129]:
df_iids = pd.DataFrame([train_ds.to_raw_iid(i) for i in train_ds.all_items()],
                       columns=['itemid'])
df_titles = pd.merge(df_iids, metadata[['itemid', 'title']], on='itemid', how='inner')
df_titles['title'].to_csv('Data/titles.tsv', sep='\t', header=False, index=False)

In [132]:
df_iids.sample()
df_titles.sample()

,itemid,title
3842,94450,얼리맨 (2018)


index 를 지정해서 순서대로 출력되도록

In [135]:
df_titles['title']

0                   시라노;연애조작단 (2010)
1                     본 얼티메이텀 (2007)
2                       어느 가족 (2018)
3                   펀치 드렁크 러브 (2002)
4                        자토이치 (2003)
5                       매그놀리아 (1999)
6                        에이리언 (1979)
7                         두더지 (2011)
8                      언브레이커블 (2000)
9                        소나티네 (1993)
10                       화양연화 (2000)
11                       동사서독 (1994)
12                       일대종사 (2012)
13                   브이 포 벤데타 (2005)
14                  네버랜드를 찾아서 (2004)
15                     플래닛 테러 (2007)
16                        클래스 (2008)
17                        천주정 (2013)
18                   사랑을 카피하다 (2010)
19                        예언자 (2009)
20               가장 따뜻한 색, 블루 (2013)
21                     스틸 라이프 (2006)
22                    퍼블릭 에너미 (2009)
23                         클랜 (2015)
24              언터처블 : 1%의 우정 (2011)
25                       아비정전 (1990)
26                         밀정 (2016)
2

In [157]:
metadata.sample(10)

,itemid,title,genres,country,running_min
5182,48702,언피니시드 (2010),스릴러/드라마,미국,113
1146,101998,맨 인 더 다크 (2016),공포/스릴러,미국,88
4103,42570,노트 온 어 스캔달 (2006),드라마,영국,-1
3182,68789,태평륜 (2014),드라마,중국,128
1642,101879,춘몽 (2016),NaN,한국,101
3221,64070,타워 하이스트 (2011),액션/코미디,미국,-1
3664,134685,브레이킹 배드 무비: 엘 카미노 (2019),드라마,미국,-1
881,3185,킬러들의 수다 (2001),코미디/드라마,한국,120
278,1829,대부 2 (1974),범죄/드라마,미국,-1
1227,69118,반창꼬 (2012),드라마/로맨스/멜로,한국,120


# 우리 데이터로 실습

In [165]:
ratings_custom = pd.read_csv('Data/dataitgirls_movie_rating.csv')

In [166]:
ratings_custom['dataitgirls'] = 1
ratings_train['dataitgirls'] = 0

In [168]:
ratings_concat = pd.concat((ratings_custom, ratings_train))

In [172]:
# build_full_trainset() : 모든 데이터를 train set 으로 쓰겠다

train_ds = Dataset.load_from_df(ratings_concat[['userid', 'itemid', 'rating']], reader).build_full_trainset()

In [173]:
model = SVD(n_factors=100, n_epochs=200, random_state=17, biased=False, )

model.fit(train_ds)

In [174]:
model

In [176]:
# 데잇걸즈 사람들만 불러오기

people = ratings_custom['userid'].unique()
people, len(people)

(array(['adela', 'bomin', 'chiwan', 'dahye', 'danbi', 'gilim', 'hansol',
        'Heeyawl', 'kwang', 'mihyeon', 'minju', 'Song', 'sunmi', 'wooju',
        'yeeun', 'yeseul'], dtype=object), 16)

In [177]:
train_ds.to_inner_uid('chiwan')

2

In [180]:
pu = []
names = []
for name in people:
    names.append(name)
    inner_uid = train_ds.to_inner_uid(name)
    pu.append(model.pu[inner_uid])

In [184]:
df_pu = pd.DataFrame(pu)

df_pu.sample()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
5,0.19611,0.848967,-0.201692,-0.663615,0.021199,0.130229,0.500319,0.028651,0.263446,-0.247247,...,0.801774,-0.014154,-0.04935,0.101638,-0.613746,-0.679318,-0.023634,0.276326,-0.364592,0.273014


In [185]:
df_names = pd.DataFrame(names)

df_names.sample()

,0
3,dahye


In [186]:
df_pu.to_csv('Data/pu_custom.tsv', sep='\t', index=False, header=False)
df_names.to_csv('Data/names_custom.tsv', sep='\t', index=False, header=False)